# LM based on convolutions

First step, reimplement https://arxiv.org/abs/1612.08083

In [19]:
# Import all needed stuff
import tensorflow as tf
import numpy as np
from IPython.display import display, HTML, clear_output
import functools
import random
from collections import namedtuple

In [20]:
print(tf.__version__)

1.8.0


In [7]:
# model parameters
params = tf.contrib.training.HParams(
    # data params
    max_string_length = 128,
    
    # training params
    batch_size = 256,
    shuffle_buffer = 100000,
    num_epochs = 10,
    
    # Model parameters
    ## Embedding
    emedding_size = 64,
    embedding_range = 0.1,
)

In [40]:
# Converts a string of text into a vector of char codes.
def _text_to_codes(text_string):
    in_array = np.array([ord(c) for c in text_string], dtype=np.uint8)
    return in_array

def convert_example(example, params=params):
    example=example.decode("utf-8")
    example = example.strip()[:params.max_string_length]
    split_pos = (len(example)*4)//5
    context = example[:split_pos]
    prediction = example[split_pos]
    return (_text_to_codes(context), _text_to_codes(prediction))


print(convert_example(b"A quick and brown fox", params))

(array([ 65,  32, 113, 117, 105,  99, 107,  32,  97, 110, 100,  32,  98,
       114, 111, 119], dtype=uint8), array([110], dtype=uint8))


In [41]:
def generateDataset(params):
    """Creates a dataset with char encoded codes"""
    dataset = tf.data.TextLineDataset("big_set.en")
    dataset = dataset.repeat(params.num_epochs)
    dataset = dataset.shuffle(params.shuffle_buffer)
    dataset = dataset.map(lambda text_string:
                          tf.py_func(functools.partial(convert_example, params=params),
                                     [text_string], [tf.uint8, tf.uint8]))
    dataset = dataset.padded_batch(params.batch_size,
                                   padded_shapes=([(params.max_string_length*4)//5], [1]))
    training_iterator = dataset.make_initializable_iterator()
    next_element = training_iterator.get_next()
    return (training_iterator, next_element)
    
    

In [ ]:
def EmbedInput(input, params, reuse_vars):
    with tf.variable_scope("embedding", reuse=reuse_vars):
        embedding = tf.get_variable("char_embedding", shape=[256,params.embedding_size],
                                   dtype=tf.float32,
                                   initializer=tf.initializers.random_uniform(minval=-params.embedding_range,
                                                                             maxval=params.embedding_range))
        embedded = tf.nn.embedding_lookup(embedding, input)
        return embedded

def GatedConvolution(input, params, reuse_vars, level_descr):
    with tf.variable_scope("gcl_%d" % level, reuse=reuse_vars):
        conv1_filters = tf.get_variable("conv1_filter", shape=[])
        conv1 = tf.nn.conv1d(input, conv1_filters, stride=1, padding="SAME")
        fully_connected1 = tf.contrib.layers.fully_connected(conv1, num_outputs=,activation_fn=None,)
    
    
    
def EncodeContext(context, params, reuse_vars):
    """Encodes context using gated convolutions https://arxiv.org/abs/1612.08083"""
    prev_values = EmbedInput(context, params, reuse_vars)
    for level_descr in params.level_descriptions:
        prev_values = Gatted
        
    

In [ ]:
with tf.Graph().as_default():
    training_iterator, next_element = generateDataset(params)
    training_flag = tf.placeholder(tf.bool, shape=(), name="training_flag")
    global_step = tf.Variable(0, trainable=False, name="global_step")
    context, prediction = next_element
    encoded = EncodeContext(context, params)
    scaffold = tf.train.Scaffold(
        local_init_op=tf.group(tf.local_variables_initializer(),training_iterator.initializer),
        init_op=tf.global_variables_initializer())
    
    with tf.train.MonitoredTrainingSession(checkpoint_dir="./checkpoint",scaffold=scaffold,
                                           save_checkpoint_secs=120) as sess:
        print("Start training")
        while not sess.should_stop():
            training_o = sess.run(next_element)
            print(training_o)
